In [ ]:
import cv2
import time
import numpy as np
import os
import math


# 각도 계산 함수
def calculate_angle(x1, y1, x2, y2):
    vector1 = (x2 - x1, y2 - y1)
    vector2 = (1, 0)
    dot_product = vector1[0] * vector2[0] + vector1[1] * vector2[1]
    magnitude1 = math.sqrt(vector1[0] ** 2 + vector1[1] ** 2)
    magnitude2 = math.sqrt(vector2[0] ** 2 + vector2[1] ** 2)
    angle = math.acos(dot_product / (magnitude1 * magnitude2))
    angle_degrees = math.degrees(angle)
    return 180 - angle_degrees

# 영상 생성
def proc(module, video):
    # Choose the model files based on the specified module
    if module == "coco":
        protoFile = "/content/drive/MyDrive/src/model/coco/pose_deploy_linevec.prototxt"
        weightsFile = "/content/drive/MyDrive/src/model/coco/pose_iter_440000.caffemodel"
        nPoints = 18
        POSE_PAIRS = [[1,0],[1,2],[1,5],[2,3],[3,4],[5,6],[6,7],[1,8],[8,9],[9,10],[1,11],[11,12],[12,13],[0,14],[0,15],[14,16],[15,17]]
    elif module == "mpi":
        # Skip processing for mpi module
        return

    # Set input size, threshold, and input video source
    inWidth = 368
    inHeight = 368
    threshold = 0.1
    input_source = "/content/drive/MyDrive/src/input/" + video

    # Open the input video file
    cap = cv2.VideoCapture(input_source)

    # Read the first frame
    hasFrame, frame = cap.read()

    # Create a VideoWriter to save the processed video
    vid_writer = cv2.VideoWriter('/content/drive/MyDrive/src/output/' + video.split(".")[0] + "_"+ module + '.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

    # Load the pre-trained pose estimation model
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

    # 추가: 각도를 저장할 리스트
    angle_list = []

    # Main loop for processing frames
    while cv2.waitKey(1) < 0:
        t = time.time()

        # Read the next frame
        hasFrame, frame = cap.read()
        frameCopy = np.copy(frame)

        # Check for the end of the video
        if not hasFrame:
            cv2.waitKey()
            break

        # Get the frame dimensions
        frameWidth = frame.shape[1]
        frameHeight = frame.shape[0]

        # Prepare the input blob for the neural network
        inpBlob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (inWidth, inHeight), (0, 0, 0), swapRB=False, crop=False)
        net.setInput(inpBlob)

        # Forward pass through the network
        output = net.forward()

        # Get the dimensions of the output map
        H = output.shape[2]
        W = output.shape[3]

        # Initialize a list to store keypoints
        points = []

        # Process each keypoint
        for i in range(nPoints):
            probMap = output[0, i, :, :]
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)
            x = (frameWidth * point[0]) / W
            y = (frameHeight * point[1]) / H

            # Check if the confidence is above the threshold
            if prob > threshold:
                # Draw a circle and label the keypoint on the frame
                cv2.circle(frameCopy, (int(x), int(y)), 8, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frameCopy, "{}".format(i), (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, lineType=cv2.LINE_AA)
                points.append((int(x), int(y)))
            else:
                points.append(None)

        # 각도 계산 및 텍스트로 그리기, 리스트에 추가
        if points[0] and points[1]:
          angle = calculate_angle(points[0][0], points[0][1], points[1][0], points[1][1])
          angle_text = "Angle: {:.2f} degrees".format(angle)
          cv2.putText(frame, angle_text, (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
          angle_list.append(angle)

        # Connect keypoints to form pose lines
        for pair in POSE_PAIRS:
            partA = pair[0]
            partB = pair[1]
            if points[partA] and points[partB]:
                cv2.line(frame, points[partA], points[partB], (0, 255, 255), 3, lineType=cv2.LINE_AA)
                cv2.circle(frame, points[partA], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)
                cv2.circle(frame, points[partB], 8, (0, 0, 255), thickness=-1, lineType=cv2.FILLED)

        # Display processing time on the frame
        cv2.putText(frame, "Test Time = {:.2f} sec".format(time.time() - t), (50, 50), cv2.FONT_HERSHEY_COMPLEX, .8, (255, 50, 0), 2, lineType=cv2.LINE_AA)

        # Write the processed frame to the output video
        vid_writer.write(frame)

    # Release the video writer
    vid_writer.release()

if __name__ == '__main__':
    # Specify the modules to use
    module = ["coco"]

    # Iterate over modules and input videos
    for i in module:
        for j in os.listdir("/content/drive/MyDrive/src/input/"):
            # Check if the file is a video (mp4 or avi)
            if j[-3:] in ['mp4','avi']:
                # Process the video
                proc(i, j)
